In [16]:

import sys 
import os 
sys.path.insert(1, '..')
sys.path.insert(2, '../modules/')
import drms
from ruffus import *
import re
import pickle
import warnings
# from create_sxi_availability_db import WORKING_DIR
warnings.filterwarnings("ignore")
from modules import convert_datetime
from modules import query_the_data
import dataconfig
from modules import sxi_module
from modules import helio_reg_exp_module
from time import sleep
from random import randint
import sqlalchemy as sa
import pandas as pd
from datetime import timedelta, date

from bs4 import BeautifulSoup
from requests import get

In [17]:
WORKING_DIR = dataconfig.DATA_DIR_GOES_SXI
DATA_PRODUCT_DIR = dataconfig.DATA_DIR_PRODUCTS

In [18]:

# some dictionaries we'll use sometimes

instruments = ['goes13', 'goes14', 'goes15']

inst_dict = {'goes13': 'g13', 'goes14': 'g14', 'goes15': 'g15'}

In [19]:

#Set timelimits you want to download from

start_download_from = '2010-05-01T00:00:00'
end_download_at = '2010-05-02T00:00:00'

START_DATE_TIME = pd.Timestamp(start_download_from, tz = 'utc')

END_DATE_TIME = pd.Timestamp(end_download_at, tz = 'utc')


In [20]:
def daterange(date1, date2):
    for n in range(int ((date2 - date1).days)+1):
        yield date1 + timedelta(n)

In [21]:
all_dates_list = []

for dt in daterange(START_DATE_TIME, END_DATE_TIME):
    # all_dates_list.append(dt.strftime("%Y%m%d"))
    all_dates_list.append(dt)

all_dates_list

[Timestamp('2010-05-01 00:00:00+0000', tz='UTC'),
 Timestamp('2010-05-02 00:00:00+0000', tz='UTC')]

In [22]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}


In [27]:
tw = lambda x: os.path.join(WORKING_DIR, x)
# Where from noaa FTP portal are you downloading from?

# ftp_prename = 'ftp://satdat.ngdc.noaa.gov/sxi/archive/fits'
http_prename = 'https://www.ncei.noaa.gov/data/goes-solar-xray-imager/access/fits'

# NOTE: we are interested in the ftp file that resembles:
#  ftp://satdat.ngdc.noaa.gov/sxi/archive/fits/goesXX/YYYY/MM/DD/*BA*.FTS

ftp_name_dict = []

for full_instrument_name in instruments:

    for this_date in all_dates_list:

        year, month, day = this_date.strftime('%Y'), this_date.strftime('%m'), this_date.strftime('%d')

        # specific_file_name = 'sci_gxrs-l2-irrad_{}_d{}_v0-0-0.nc'.format(inst_dict[full_instrument_name], specific_date_str)

        ftp_name = f'{http_prename}/{full_instrument_name}/{year}/{month}/{day}'

        http_name = f'{http_prename}/{full_instrument_name}/{year}/{month}/{day}'

        outfile_str = tw(f'{year}-{month}-{day}_{full_instrument_name}')

        ftp_name_dict.append({'availability_file': http_name , 'out_name': outfile_str })



In [28]:
ftp_name_dict

[{'availability_file': 'https://www.ncei.noaa.gov/data/goes-solar-xray-imager/access/fits/goes13/2010/05/01',
  'out_name': '/mnt/e/ALEXIS_data/sxi_availability/2010-05-01_goes13'},
 {'availability_file': 'https://www.ncei.noaa.gov/data/goes-solar-xray-imager/access/fits/goes13/2010/05/02',
  'out_name': '/mnt/e/ALEXIS_data/sxi_availability/2010-05-02_goes13'},
 {'availability_file': 'https://www.ncei.noaa.gov/data/goes-solar-xray-imager/access/fits/goes14/2010/05/01',
  'out_name': '/mnt/e/ALEXIS_data/sxi_availability/2010-05-01_goes14'},
 {'availability_file': 'https://www.ncei.noaa.gov/data/goes-solar-xray-imager/access/fits/goes14/2010/05/02',
  'out_name': '/mnt/e/ALEXIS_data/sxi_availability/2010-05-02_goes14'},
 {'availability_file': 'https://www.ncei.noaa.gov/data/goes-solar-xray-imager/access/fits/goes15/2010/05/01',
  'out_name': '/mnt/e/ALEXIS_data/sxi_availability/2010-05-01_goes15'},
 {'availability_file': 'https://www.ncei.noaa.gov/data/goes-solar-xray-imager/access/fits/

In [ ]:
output_df = []
for this_dict in ftp_name_dict:

    # print('start')

    # print(this_dict)

    """ Ask for a webresponse and parse throught the available links """

    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}

    queries = this_dict

    http_query_name = queries['availability_file']

    response = get(http_query_name, headers = headers)
    # print(f'{response.status_code} for {http_query_name}')

    # Pause the loop
    # sleep(randint(4,37))

        # ADD LONG SLEEP AFTER 429 AND ALSO 
        # RE-TRY LOOP FOR 429 ==> MAYBE

        # Throw a warning for non-200 status codes
    if response.status_code == 429:

        print('{} w/ Status code: {}'.format(http_query_name, response.status_code))
        print(BeautifulSoup(response.content, 'html.parser'))
        # break

    if response.status_code == 200:
        html_soup = BeautifulSoup(response.content, 'html.parser')

        sxi_links = []

        for link in html_soup.find_all('a'):

            file_name = str(link.get('href'))

            if file_name.startswith('SXI'):

                level= re.findall(r'(?<=\d{9}_)[A-Za-z0-9]{2}', file_name)[0]

                inst = re.findall(r'(?<=\d{9}_[A-Za-z0-9]{2}_)\d{2}', file_name)[0]

                date_time_str = re.findall(r'\d{8}_\d{9}', file_name )[0]

                date_time_obj = pd.to_datetime(date_time_str, format = '%Y%m%d_%H%M%S%f', utc = True)

                time_stamp = convert_datetime.convert_datetime_to_timestamp(date_time_obj)

                year, month, day = date_time_obj.strftime('%Y'), date_time_obj.strftime('%m'), date_time_obj.strftime('%d')

                full_instrument_name = f'goes{inst}'

                # ftp_prename = 'ftp://satdat.ngdc.noaa.gov/sxi/archive/fits'

                http_query_name = f'{http_prename}/{full_instrument_name}/{year}/{month}/{day}/{file_name}'

                download_str = f'wget -e robots=off --recursive --no-parent globs = True --directory-prefix {WORKING_DIR} --no-directories --verbose False {http_query_name}'

                sxi_links.append({'url': http_query_name, 'download_string': download_str, 'time_stamp': time_stamp, 'data_level': level, 'file_name': file_name, 'instrument': full_instrument_name})

        available_data_df = pd.DataFrame(sxi_links)

        output_df.append(available_data_df)


        # print(available_data_df)

        # pickle.dump(available_data_df, open(outfile, 'wb'))

        # print('------------------------------')
    
    if response.status_code == 404:
        no_data = pd.DataFrame()
        print(f'no data for {http_query_name}')

        # pickle.dump(no_data, open(outfile, 'wb'))

no data for https://www.ncei.noaa.gov/data/goes-solar-xray-imager/access/fits/goes13/2010/05/01
no data for https://www.ncei.noaa.gov/data/goes-solar-xray-imager/access/fits/goes13/2010/05/02
no data for https://www.ncei.noaa.gov/data/goes-solar-xray-imager/access/fits/goes15/2010/05/01
no data for https://www.ncei.noaa.gov/data/goes-solar-xray-imager/access/fits/goes15/2010/05/02


In [37]:
this_df = pd.concat(output_df)
this_df

,url,download_string,time_stamp,data_level,file_name,instrument
0,https://www.ncei.noaa.gov/data/goes-solar-xray...,wget -e robots=off --recursive --no-parent glo...,1.272672e+09,AA,SXI_20100501_000015157_AA_14.FTS,goes14
1,https://www.ncei.noaa.gov/data/goes-solar-xray...,wget -e robots=off --recursive --no-parent glo...,1.272672e+09,AA,SXI_20100501_000115163_AA_14.FTS,goes14
2,https://www.ncei.noaa.gov/data/goes-solar-xray...,wget -e robots=off --recursive --no-parent glo...,1.272672e+09,AA,SXI_20100501_000215144_AA_14.FTS,goes14
3,https://www.ncei.noaa.gov/data/goes-solar-xray...,wget -e robots=off --recursive --no-parent glo...,1.272672e+09,AA,SXI_20100501_000315165_AA_14.FTS,goes14
4,https://www.ncei.noaa.gov/data/goes-solar-xray...,wget -e robots=off --recursive --no-parent glo...,1.272672e+09,AA,SXI_20100501_000400165_AA_14.FTS,goes14
...,...,...,...,...,...,...
1795,https://www.ncei.noaa.gov/data/goes-solar-xray...,wget -e robots=off --recursive --no-parent glo...,1.272845e+09,AA,SXI_20100502_235615146_AA_14.FTS,goes14
1796,https://www.ncei.noaa.gov/data/goes-solar-xray...,wget -e robots=off --recursive --no-parent glo...,1.272845e+09,AA,SXI_20100502_235700154_AA_14.FTS,goes14
1797,https://www.ncei.noaa.gov/data/goes-solar-xray...,wget -e robots=off --recursive --no-parent glo...,1.272845e+09,AA,SXI_20100502_235715148_AA_14.FTS,goes14
1798,https://www.ncei.noaa.gov/data/goes-solar-xray...,wget -e robots=off --recursive --no-parent glo...,1.272845e+09,AA,SXI_20100502_235815138_AA_14.FTS,goes14


In [38]:
this_df.iloc[0].download_string

'wget -e robots=off --recursive --no-parent globs = True --directory-prefix /mnt/e/ALEXIS_data/sxi_availability --no-directories --verbose False https://www.ncei.noaa.gov/data/goes-solar-xray-imager/access/fits/goes14/2010/05/01/SXI_20100501_000015157_AA_14.FTS'

In [39]:
os.system(this_df.iloc[0].download_string)

1024

In [41]:
os.path.exists(f'{WORKING_DIR}/{this_df.iloc[0].file_name}')

True

In [ ]:
# code above works and below is exploratory

In [13]:
test_https_pre = 'https://www.ncei.noaa.gov/data/goes-solar-xray-imager/access/fits/goes13/2015/07/07'

In [14]:
test_https_link = f'{test_https_pre}/SXI_20150707_000015167_BA_13.FTS'
test_https_link

'https://www.ncei.noaa.gov/data/goes-solar-xray-imager/access/fits/goes13/2015/07/07/SXI_20150707_000015167_BA_13.FTS'

In [15]:
response = get(test_https_link, headers = headers)
response.status_code

200

In [5]:
download_str = f'wget -e robots=off --recursive --no-parent globs = True --directory-prefix {WORKING_DIR} --no-directories --verbose False {test_https_link}'


In [6]:
download_str

'wget -e robots=off --recursive --no-parent globs = True --directory-prefix /mnt/e/ALEXIS_data/sxi_availability --no-directories --verbose False https://ftp.ngdc.noaa.gov/data/goes-solar-xray-imager/access/fits/goes13/2015/07/07/SXI_20150707_000015167_BA_13.FTS'

In [ ]:
os.system(download_str)

In [9]:
ftp_prename = 'ftp://satdat.ngdc.noaa.gov/sxi/archive/fits'

full_instrument_name='goes13'
year='2015'
month='07'
day='07'

file_name = 'SXI_20150707_000015167_BA_13.FTS'

ftp_query_name = f'{ftp_prename}/{full_instrument_name}/{year}/{month}/{day}/{file_name}'

download_str = f'wget -e robots=off --recursive --no-parent globs = True --directory-prefix {WORKING_DIR} --no-directories --verbose False {ftp_query_name}'

In [10]:
download_str

'wget -e robots=off --recursive --no-parent globs = True --directory-prefix /mnt/e/ALEXIS_data/sxi_availability --no-directories --verbose False ftp://satdat.ngdc.noaa.gov/sxi/archive/fits/goes13/2015/07/07/SXI_20150707_000015167_BA_13.FTS'

In [11]:
os.system(download_str)

1024